In [1]:
# 导入包
import pandas as pd

C:\Users\ASUS\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ASUS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\ASUS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def load_Data(path):
    """加载数据"""
    Data = pd.read_csv(path, nrows=10000)
    Ratings = Data.pivot_table(index='userId', columns='movieId', values='rating') #将数据转换成用户-评分矩阵
    return Ratings

In [3]:
path = r'G:\github项目\Recomendation_system\Data\movie\ratings.csv'
UserItemMatrix = load_Data(path) # 加载数据

In [4]:
UserItemMatrix.head()  # 简单的观察用户-评分矩阵，存在许多空值

movieId,1,2,3,4,5,6,7,8,10,11,...,182715,183611,184471,185031,185135,187541,187593,187595,188301,190183
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def person_similarity(UserItemMatrix, based='user'):
    """计算皮尔逊相关系数，这里是基于用户的UserCF，若计算ItemCF可改为item"""
    if based == 'user':
        similarity = UserItemMatrix.T.corr(method='pearson') # corr是按列计算相关性，故这里计算用户相关性需要将用户矩阵转置
    else:
        similarity = UserItemMatrix.corr(method='pearson')
    return similarity
UserSimilarity = person_similarity(UserItemMatrix)  # 生成用户-评分矩阵

In [6]:
UserSimilarity.head()  # 得到各用户的相关系数

userId,1,2,3,4,5,6,7,8,9,10,...,57,58,59,60,61,62,63,64,65,66
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,NaN,0.079819,0.207983,0.268749,-0.291636,-0.118773,0.469668,0.918559,-0.037987,...,0.354260,0.351998,7.534646e-02,NaN,0.156315,0.511682,0.284589,0.201428,0.612372,0.000000
2,NaN,1.0,NaN,NaN,NaN,NaN,-0.991241,NaN,NaN,0.037796,...,-1.000000,NaN,NaN,1.0,-1.000000,-0.162224,-0.307897,-0.366900,0.121367,NaN
3,0.079819,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.675600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.207983,NaN,NaN,1.000000,-0.336525,0.148498,0.542861,0.117851,NaN,0.485794,...,0.252681,0.154437,3.347762e-01,-0.5,-0.683130,-0.090781,0.222954,0.032385,NaN,-0.997509
5,0.268749,NaN,NaN,-0.336525,1.000000,0.043166,0.158114,0.028347,NaN,-0.777714,...,0.288799,0.482395,5.417334e-17,0.5,0.654654,0.559893,0.046944,0.107369,NaN,NaN


In [8]:
def select_similar_user(UserSimilarity, k, uid):
    """根据用户相似度返回与指定用户最相似的前TopK个用户"""
    UserSimilarity = UserSimilarity.loc[uid].dropna()  # 挑出指定用户，并排除空值
    SimilarUser = []  # 用于存储候选的相似用户
    
    for Uid in UserSimilarity.index:
        if not pd.isnull(UserSimilarity.loc[Uid]):
            SimilarUser.append([Uid, UserSimilarity.loc[Uid]])
            
    SimilarUser.sort(key=lambda x: -x[1]) # 按照相似度降序排序
    SimilarUser = [SimilarUser[i][0] for i in range(len(SimilarUser))]  # 选出排序后的id
    return SimilarUser[:k]
select_similar_user(UserSimilarity, 20, 1)

[1, 9, 13, 49, 44, 65, 55, 62, 47, 8, 36, 20, 57, 58, 19, 35, 63, 32, 5, 45]

In [9]:
def user_likemovie(UserItemMatrix, uid):
    """返回指定用户评过分的电影Id"""
    like_movies = []
    movies = UserItemMatrix.loc[uid].dropna()
    for i in movies.index:
        like_movies.append([i, movies.loc[i]]) # 得到指定用户评过分的所有电影
    like_movies.sort(key=lambda x: -x[1])  # 按评分降序排序
    like_movies = [like_movies[i][0] for i in range(len(like_movies))]
    return list(like_movies)
like_movies = user_likemovie(UserItemMatrix, 1)

In [10]:
def recommend_movie(uid, UserItemMatrix, k, UserSimilarity):
    """给指定用户推荐其未看过的电影，根据用户相似度进行推荐"""
    similar_users = select_similar_user(UserSimilarity, k, uid)
    moviesHold = []
    
    for similar_user in similar_users:
        moviesHold.append(user_likemovie(UserItemMatrix, similar_user)) # 添加待选的电影
    
    selfs_movies = list(UserItemMatrix.loc[uid].dropna().index) # 选出自己看过的电影
    
    movies_count = {}
    recommend_movies = []
    for movies in moviesHold:
        for movie in movies:
            if movie not in movies_count:
                movies_count[movie] = 1
            else:
                movies_count[movie] += 1
            
    for movie in movies_count:
        if movies_count[movie] > k // 3:
            recommend_movies.append(movie) # 若相似的电影推荐数量超过相似用户的1/3，便将其推荐给该用户
    
    return list(set(recommend_movies) & set(selfs_movies))  # 取推荐电影和自己看过电影的交集
recommend_movie(1, UserItemMatrix, 10, UserSimilarity)

[356, 1097, 3147, 2571, 2028, 1198, 527, 47, 2959, 110, 593]

In [16]:
def predict_score(uid, iid, UserItemMatrix, similarity):
    """预测给定用户对给定用户的评分"""
    similar_users = similarity.loc[uid].drop([uid]).dropna()  # 排除掉自身
    similar_users = similar_users.where(similar_users > 0).dropna()  # 选相似度大于0的进行计算
    indexes = set(UserItemMatrix[iid].dropna().index) & set(similar_users.index) 
    similar_users = similar_users.loc[list(indexes)]
    
    sum_up = 0
    sum_down = 0
    for sim_uid, similarity in similar_users.iteritems():
        sim_user_similarity_movie = UserItemMatrix.loc[sim_uid]
        sim_user_rating_for_item = sim_user_similarity_movie[iid]
        sum_up += similarity * sim_user_rating_for_item
        sum_down += similarity
    return sum_up / (sum_down + 1e-8)  # 防止分母为0
predict_score(1, 1, UserItemMatrix, UserSimilarity)  # 预测uid为1对iid为1的评分，实际为4

3.7711029123966044

In [17]:
def predict_all(UserItemMatrix, similarity):
    """预测全部用户对物品的评分"""
    user_id = UserItemMatrix.index
    item_id = UserItemMatrix.columns
    Predict_Matrix = pd.DataFrame(index=user_id, columns=item_id) # 新建一个与原先用户评分矩阵相同的空矩阵
    
    # 记录误差和实际值
    error = 0
    actual_value = 0
    
    for uid in user_id:
        for iid in item_id:
            Predict_Matrix.loc[uid].loc[iid] = predict_score(uid, iid, UserItemMatrix, similarity)
            if not pd.isnull(UserItemMatrix.loc[uid].loc[iid]):
                error += abs(UserItemMatrix.loc[uid].loc[iid] - Predict_Matrix.loc[uid].loc[iid])  # 按照已有的评分进行准确率评估
                actual_value += UserItemMatrix.loc[uid].loc[iid]
    
    # 计算准确率
    accuracy = error / actual_value
    return Predict_Matrix, accuracy
Pre, acc = predict_all(UserItemMatrix, UserSimilarity)

In [18]:
Pre.head()

movieId,1,2,3,4,5,6,7,8,10,11,...,182715,183611,184471,185031,185135,187541,187593,187595,188301,190183
userId,,,,,,,,,,,,,,,,,,,,,
1,3.771103,3.576519,3.245341,3.0,3.752012,3.696564,3.423272,1.83846,3.07044,3.81575,...,4.0,4.0,3.5,4.5,4.856795,0.0,4.0,4.0,0.0,0.0
2,3.333333,4.5,4.0,0.0,3.0,4.0,2.648612,0.0,5.0,2.832985,...,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0
3,3.999999,0.0,3.999999,0.0,0.0,3.640547,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.914093,3.346609,3.833293,3.0,4.778118,3.672263,4.100876,3.308228,3.425029,3.940897,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.5
5,4.388686,3.570757,3.60821,2.999999,3.521608,3.993805,3.622662,4.221456,3.535705,3.93026,...,4.0,4.0,3.5,4.5,4.887261,0.0,4.0,4.0,0.0,0.0


In [19]:
acc

0.36777454934549364